### RANDOMCIRCUIT SIZE 8,16,32 IN IBM (Falcon) AND QUANTINUUM

In [1]:
# Caller
from pathlib import Path
import sys

# Discover project root
proj = Path.cwd().resolve()
for _ in range(8):
    if (proj / "results" / "pairwise" / "summary").exists() and (proj / "notebooks").exists():
        break
    if proj.parent == proj:
        break
    proj = proj.parent

# Import helper
sys.path.insert(0, str(proj / "notebooks"))
from notebook_helper import load_family, tidy, show_table  # (sort_best_worst, add_rank_within_size available if needed)

# analysis parameters
FAMILY = "randomcircuit"  # or: "qaoa","vqe_two_local","qft","grover","vbe_adder","randomcircuit"
BACKENDS = ("ibm_falcon","quantinuum")

loaded = load_family(FAMILY, BACKENDS)

for backend, df in loaded.items():
    tbl = tidy(df)
    show_table(tbl, caption=f"{FAMILY.upper()} – {backend} (raw tidy)")

    # To rank later, uncomment:
    # from notebook_helper import sort_best_worst, add_rank_within_size
    # PRIMARY_METRIC = "depth_delta"  # or "n_ops_total_gates_delta", "n_ops_total_delta", etc.
    # ranked = sort_best_worst(tbl, primary=PRIMARY_METRIC, per_size=True)
    # ranked = add_rank_within_size(ranked, by=PRIMARY_METRIC)
    # show_table(ranked, caption=f"{FAMILY.upper()} – {backend} (ranked by {PRIMARY_METRIC})")

,backend,family,size,variant,sequence,depth_before,depth_after,depth_delta,n_ops_total_before,n_ops_total_after,n_ops_total_delta,n_ops_total_gates_before,n_ops_total_gates_after,n_ops_total_gates_delta,n_ops1_before,n_ops1_after,n_ops1_delta,n_ops2_before,n_ops2_after,n_ops2_delta,other_before,other_after,other_delta,barrier_before,barrier_after,barrier_delta,measure_before,measure_after,measure_delta,reset_before,reset_after,reset_delta,top_ops_before,top_ops_after,n_qubits_before,n_qubits_after,timestamp
0,ibm_falcon,randomcircuit,16,-,RB→RR,761,642,-119,3546,3072,-474,3529,3056,-473,2614,2141,-473,931,931,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 2065], [""cx"", 931], [""sx"", 529], [""measure"", 16], [""x"", 3], [""noop"", 1], [""barrier"", 1]]","[[""rz"", 1593], [""cx"", 931], [""sx"", 529], [""measure"", 16], [""x"", 3]]",16,16,2025-10-06T01:11:32
1,ibm_falcon,randomcircuit,16,-,RR→RB,761,643,-118,3546,3075,-471,3529,3059,-470,2614,2144,-470,931,931,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 2065], [""cx"", 931], [""sx"", 529], [""measure"", 16], [""x"", 3], [""noop"", 1], [""barrier"", 1]]","[[""rz"", 1596], [""cx"", 931], [""sx"", 529], [""measure"", 16], [""x"", 3]]",16,16,2025-10-06T01:11:32
2,ibm_falcon,randomcircuit,16,-,RB→CTM,761,776,+15,3546,3545,-1,3529,3529,+0,2614,2614,+0,931,931,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 2065], [""cx"", 931], [""sx"", 529], [""measure"", 16], [""x"", 3], [""noop"", 1], [""barrier"", 1]]","[[""rz"", 2065], [""cx"", 931], [""sx"", 529], [""measure"", 16], [""x"", 3], [""noop"", 1]]",16,16,2025-10-06T01:11:32
3,ibm_falcon,randomcircuit,16,-,CTM→RB,761,776,+15,3546,3545,-1,3529,3529,+0,2614,2614,+0,931,931,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 2065], [""cx"", 931], [""sx"", 529], [""measure"", 16], [""x"", 3], [""noop"", 1], [""barrier"", 1]]","[[""rz"", 2065], [""cx"", 931], [""sx"", 529], [""measure"", 16], [""x"", 3], [""noop"", 1]]",16,16,2025-10-06T01:11:32
4,ibm_falcon,randomcircuit,16,-,RR→CTM,761,652,-109,3546,3076,-470,3529,3059,-470,2614,2144,-470,931,931,+0,1,1,+0,1,1,+0,16,16,+0,0,0,+0,"[[""rz"", 2065], [""cx"", 931], [""sx"", 529], [""measure"", 16], [""x"", 3], [""noop"", 1], [""barrier"", 1]]","[[""rz"", 1596], [""cx"", 931], [""sx"", 529], [""measure"", 16], [""x"", 3], [""barrier"", 1]]",16,16,2025-10-06T01:11:32
5,ibm_falcon,randomcircuit,16,-,CTM→RR,761,631,-130,3546,2894,-652,3529,2877,-652,2614,1962,-652,931,931,+0,1,1,+0,1,1,+0,16,16,+0,0,0,+0,"[[""rz"", 2065], [""cx"", 931], [""sx"", 529], [""measure"", 16], [""x"", 3], [""noop"", 1], [""barrier"", 1]]","[[""rz"", 1414], [""cx"", 931], [""sx"", 529], [""measure"", 16], [""x"", 3], [""barrier"", 1]]",16,16,2025-10-06T01:11:32
6,ibm_falcon,randomcircuit,32,-,RB→RR,2110,1753,-357,14278,12336,-1942,14245,12304,-1941,10549,8610,-1939,3728,3726,-2,1,0,-1,1,0,-1,32,32,+0,0,0,+0,"[[""rz"", 8351], [""cx"", 3728], [""sx"", 2138], [""measure"", 32], [""x"", 19], [""noop"", 9], [""barrier"", 1]]","[[""rz"", 6421], [""cx"", 3726], [""sx"", 2138], [""measure"", 32], [""x"", 19]]",32,32,2025-10-06T01:11:33
7,ibm_falcon,randomcircuit,32,-,RR→RB,2110,1754,-356,14278,12346,-1932,14245,12314,-1931,10549,8620,-1929,3728,3726,-2,1,0,-1,1,0,-1,32,32,+0,0,0,+0,"[[""rz"", 8351], [""cx"", 3728], [""sx"", 2138], [""measure"", 32], [""x"", 19], [""noop"", 9], [""barrier"", 1]]","[[""rz"", 6431], [""cx"", 3726], [""sx"", 2138], [""measure"", 32], [""x"", 19]]",32,32,2025-10-06T01:11:34
8,ibm_falcon,randomcircuit,32,-,RB→CTM,2110,2108,-2,14278,14277,-1,14245,14245,+0,10549,10549,+0,3728,3728,+0,1,0,-1,1,0,-1,32,32,+0,0,0,+0,"[[""rz"", 8351], [""cx"", 3728], [""sx"", 2138], [""measure"", 32], [""x"", 19], [""noop"", 9], [""barrier"", 1]]","[[""rz"", 8351], [""cx"", 3728], [""sx"", 2138], [""measure"", 32], [""x"", 19], [""noop"", 9]]",32,32,2025-10-06T01:11:35
9,ibm_falcon,randomcircuit,32,-,CTM→RB,2110,2108,-2,14278,14277,-1,14245,14245,+0,10549,10549,+0,3728,3728,+0,1,0,-1,1,0,-1,32,32,+0,0,0,+0,"[[""rz"", 8351], [""cx"", 

,backend,family,size,variant,sequence,depth_before,depth_after,depth_delta,n_ops_total_before,n_ops_total_after,n_ops_total_delta,n_ops_total_gates_before,n_ops_total_gates_after,n_ops_total_gates_delta,n_ops1_before,n_ops1_after,n_ops1_delta,n_ops2_before,n_ops2_after,n_ops2_delta,other_before,other_after,other_delta,barrier_before,barrier_after,barrier_delta,measure_before,measure_after,measure_delta,reset_before,reset_after,reset_delta,top_ops_before,top_ops_after,n_qubits_before,n_qubits_after,timestamp
0,quantinuum,randomcircuit,8,-,RB→RR,523,473,-50,1545,1362,-183,1536,1354,-182,1364,1182,-182,180,180,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rx"", 658], [""ry"", 381], [""rz"", 317], [""zzphase"", 180], [""measure"", 8], [""barrier"", 1]]","[[""rx"", 487], [""ry"", 381], [""rz"", 306], [""zzphase"", 180], [""measure"", 8]]",8,8,2025-10-06T01:11:38
1,quantinuum,randomcircuit,8,-,RR→RB,523,473,-50,1545,1367,-178,1536,1359,-177,1364,1187,-177,180,180,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rx"", 658], [""ry"", 381], [""rz"", 317], [""zzphase"", 180], [""measure"", 8], [""barrier"", 1]]","[[""rx"", 487], [""ry"", 381], [""rz"", 311], [""zzphase"", 180], [""measure"", 8]]",8,8,2025-10-06T01:11:38
2,quantinuum,randomcircuit,8,-,RB→CTM,523,539,+16,1545,1544,-1,1536,1536,+0,1364,1364,+0,180,180,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rx"", 658], [""ry"", 381], [""rz"", 317], [""zzphase"", 180], [""measure"", 8], [""barrier"", 1]]","[[""rx"", 658], [""ry"", 381], [""rz"", 317], [""zzphase"", 180], [""measure"", 8]]",8,8,2025-10-06T01:11:38
3,quantinuum,randomcircuit,8,-,CTM→RB,523,539,+16,1545,1544,-1,1536,1536,+0,1364,1364,+0,180,180,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rx"", 658], [""ry"", 381], [""rz"", 317], [""zzphase"", 180], [""measure"", 8], [""barrier"", 1]]","[[""rx"", 658], [""ry"", 381], [""rz"", 317], [""zzphase"", 180], [""measure"", 8]]",8,8,2025-10-06T01:11:38
4,quantinuum,randomcircuit,8,-,RR→CTM,523,483,-40,1545,1368,-177,1536,1359,-177,1364,1187,-177,180,180,+0,1,1,+0,1,1,+0,8,8,+0,0,0,+0,"[[""rx"", 658], [""ry"", 381], [""rz"", 317], [""zzphase"", 180], [""measure"", 8], [""barrier"", 1]]","[[""rx"", 487], [""ry"", 381], [""rz"", 311], [""zzphase"", 180], [""measure"", 8], [""barrier"", 1]]",8,8,2025-10-06T01:11:38
5,quantinuum,randomcircuit,8,-,CTM→RR,523,463,-60,1545,1296,-249,1536,1287,-249,1364,1115,-249,180,180,+0,1,1,+0,1,1,+0,8,8,+0,0,0,+0,"[[""rx"", 658], [""ry"", 381], [""rz"", 317], [""zzphase"", 180], [""measure"", 8], [""barrier"", 1]]","[[""rx"", 487], [""ry"", 381], [""rz"", 239], [""zzphase"", 180], [""measure"", 8], [""barrier"", 1]]",8,8,2025-10-06T01:11:38
6,quantinuum,randomcircuit,32,-,RB→RR,2363,2130,-233,26855,23621,-3234,26822,23589,-3233,23773,20540,-3233,3081,3081,+0,1,0,-1,1,0,-1,32,32,+0,0,0,+0,"[[""rx"", 11442], [""ry"", 6780], [""rz"", 5519], [""zzphase"", 3081], [""measure"", 32], [""barrier"", 1]]","[[""rx"", 8385], [""ry"", 6754], [""rz"", 5369], [""zzphase"", 3081], [""measure"", 32]]",32,32,2025-10-06T01:11:40
7,quantinuum,randomcircuit,32,-,RR→RB,2363,2130,-233,26855,23636,-3219,26822,23604,-3218,23773,20555,-3218,3081,3081,+0,1,0,-1,1,0,-1,32,32,+0,0,0,+0,"[[""rx"", 11442], [""ry"", 6780], [""rz"", 5519], [""zzphase"", 3081], [""measure"", 32], [""barrier"", 1]]","[[""rx"", 8385], [""ry"", 6754], [""rz"", 5384], [""zzphase"", 3081], [""measure"", 32]]",32,32,2025-10-06T01:11:41
8,quantinuum,randomcircuit,32,-,RB→CTM,2363,2490,+127,26855,26854,-1,26822,26822,+0,23773,23773,+0,3081,3081,+0,1,0,-1,1,0,-1,32,32,+0,0,0,+0,"[[""rx"", 11442], [""ry"", 6780], [""rz"", 5519], [""zzphase"", 3081], [""measure"", 32], [""barrier"", 1]]","[[""rx"", 11442], [""ry"", 6780], [""rz"", 5519], [""zzphase"", 3081], [""measure"", 32]]",32,32,2025-10-06T01:11:42
9,quantinuum,randomcircuit,32,-,CTM→RB,2363,2490,+127,26855,26854,-1,26822,26822,+0,23773,23773,+0,3081,3081,+0,1,0,-1,1,0,-1,32,32,+0,0,0,+0,"[[""rx"", 11442], [""ry"", 6780], [""rz"", 5519], [""zzphase"", 3081], [""measure"", 32], [""barrier"", 1]]","

### quick findings

### ibm
### randomcirc size 8
- **RB ↔ RR:** both reduce depth; **RB → RR** cuts more by 1 (−51 vs −50) total-ops and gate reductions follow the same pattern; **all savings are from 1-qubit gates**. **2-qubit unchanged. Barrier removed.**
- **CTM ↔ RB:** **both add depth +1** and remove barrier. No other meaningful metric changes.
- **RR ↔ CTM:** both cut depth, with **CTM → RR** leading. Ops and gate counts fall in the same direction as depth
- 
### randomcirc size 16
- **RB ↔ RR:** same behaviour
- **CTM ↔ RB:** same behaviour
- **RR ↔ CTM:** same behaviour

### randomcirc size 32
- **RB ↔ RR:** same behaviour,small reduction in 2-qubit counts (−2) observed here
- **CTM ↔ RB:** depth −1
- **CTM ↔ RR:** same behaviour,small reduction in 2-qubit counts (−2) observed here

---

### quantinuum

### randomcirc size 8
- **RB ↔ RR:** equal depth reduction in both directions (no directional advantage)
- **RB ↔ CTM:** **adds depth (+16)** in both directions, removes barrier, nothing else changes
- **RR ↔ CTM:** depth reduced with **CTM → RR** leading, ops/gates reduced similarly

### randomcirc size 16
- Same behaviour as size 8, **but** note: **RB → RR** shows a **−1** change in 2-qubit count (small 2-qubit reduction observed), otherwise the same patterns

### randomcirc size 32
- Same behaviour as other sizes; numbers differ but patterns hold

---

### summary
- **Main driver:** **1-qubit gate cancellations** explain nearly all Δdepth/Δops
- **Directional effects:** When a directional advantage exists it is small (typically ~1 depth) but consistent:
  - **RB → RR** often edges out RR → RB
  - **CTM → RR** tends to enable larger improvements than the reverse
- **CTM pairing:** CTM pairs **poorly with RB** (neutral or harmful), but **helps** when applied *before* RR.